In [1]:
import numpy as np
import pandas as pd
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
#from xgboost import XGBRegressor
#from xgboost import plot_importance
import matplotlib.pyplot as plt
from matplotlib import rc
import statsmodels.api as sm
import json
from rh2q_2d import *
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.transforms import transforms
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader, TensorDataset
iopath='/content/drive/MyDrive/THEQC/'

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
def t2the(p,t,q):
    Lv=2.5e6
    Cp=1004.
    Rd=287.
    th=(t+273.15)*((1000./p)**(Rd/Cp))
    the=th*np.exp((Lv*(q/1000.))/(Cp*(t+273.15)))

    return th, the

In [3]:
def data_split(nday):
    # Split the dataset: 80% for training and 20% for test
    fsize=np.arange(nday)
    ftrain0, ftest0=train_test_split(fsize, random_state=777, train_size=0.8)
    ftrain=sorted(ftrain0)
    ftest=sorted(ftest0)

    return ftrain, ftest

In [4]:
def preprocess(data,label):
  if label=='st':
    p1=data[0,:,:]/1100.
    t1=(data[1,:,:]-(-50.))/100.
    rh1=data[2,:,:]/100.
    u1=(data[3,:,:]-(-50.))/100.
    v1=(data[4,:,:]-(-50.))/100.
    rad1=data[5,:,:]/1400.
    q1=data[6,:,:]/data[7,:,:]
    th1=(data[8,:,:]-300.)/100.
    the1=(data[9,:,:]-300.)/100.
    st_data=np.array([p1,t1,rh1,u1,v1,rad1,q1,th1,the1],dtype='float')
    return st_data
  elif label=='vs':
    p1=data[0,:,:]/1100.
    t1=(data[1,:,:]-(-50.))/100.
    rh1=data[2,:,:]/100.
    q1=data[3,:,:]/data[4,:,:]
    th1=(data[5,:,:]-300.)/100.
    the1=(data[6,:,:]-300.)/100.
    vs_data=np.array([p1,t1,rh1,q1,th1,the1],dtype='float')
    return vs_data

In [5]:
def load_data(stpath,vspath):
    #ST
    stinf=pd.read_csv(stpath)
    nlev=701
    nrow=stinf.shape[0]
    nday=int(nrow/nlev)
    date=stinf['date'].astype('int').values.reshape(nday,nlev)
    p_st=stinf['P'].values.reshape(nday,nlev)
    t_st=stinf['T'].values.reshape(nday,nlev)
    rh_st=stinf['RH'].values.reshape(nday,nlev)
    u_st=stinf['U'].values.reshape(nday,nlev)
    v_st=stinf['V'].values.reshape(nday,nlev)
    rad_st=stinf['rad'].values.reshape(nday,nlev)
    t_diff_st=stinf['T_diff'].values.reshape(nday,nlev)
    rh_diff_st=stinf['RH_diff'].values.reshape(nday,nlev)
    # Calculate specific humidity/theta, thetae
    q_st, q0_st=rh_to_q(p_st,t_st,rh_st)
    q_st=stinf['q'].values.reshape(nday,nlev)
    th_st, the_st=t2the(p_st,t_st,q_st)

    # VS
    vsinf=pd.read_csv(vspath)
    p_vs=vsinf['P'].values.reshape(nday,nlev)
    t_vs=vsinf['T'].values.reshape(nday,nlev)
    rh_vs=vsinf['RH'].values.reshape(nday,nlev)
    # Calculate specific humidity/theta, thetae
    q_vs, q0_vs=rh_to_q(p_vs,t_vs,rh_vs)
    q_vs=vsinf['q'].values.reshape(nday,nlev)
    th_vs, the_vs=t2the(p_vs,t_vs,q_vs)

    # Calculate q/th difference
    q_diff=q_vs-q_st
    th_diff=th_vs-th_st
    the_diff=the_vs-the_st

    st_data=np.array([p_st,t_st,rh_st,u_st,v_st,rad_st,q_st,q0_st,th_st,the_st],dtype='float')
    vs_data=np.array([p_vs,t_vs,rh_vs,q_vs,q0_vs,th_vs,the_vs],dtype='float')

    st_input=preprocess(st_data,'st')
    vs_input=preprocess(vs_data,'vs')

    id_train, id_test=data_split(nday)
    st_train=st_input[:,id_train,:]
    st_test=st_input[:,id_test,:]

    np.save(iopath+'st_train.npy',st_train)
    np.save(iopath+'st_test.npy',st_test)
    vs_train=vs_input[:,id_train,:]
    vs_test=vs_input[:,id_test,:]
    np.save(iopath+'vs_train.npy',vs_train)
    np.save(iopath+'vs_test.npy',vs_test)
    the_diff_train=the_diff[id_train,:]
    the_diff_test=the_diff[id_test,:]
    date_train=date[id_train,:]
    date_test=date[id_test,:]

    return st_train, st_test, vs_train, vs_test, the_diff_train, the_diff_test,date_train,date_test

In [6]:
def nd21d(st3d,vs3d,the2d):
    st2d=st3d.reshape(st3d.shape[0],st3d.shape[1]*st3d.shape[2])
    vs2d=vs3d.reshape(vs3d.shape[0],vs3d.shape[1]*vs3d.shape[2])
    the1d_0=the2d.reshape(the2d.shape[0]*the2d.shape[1])
    #st_data=np.array([p1,t1,rh1,u1,v1,rad1,q1,th1,the1],dtype='float')
    #vs_data=np.array([p1,t1,rh1,q1,th1,the1],dtype='float')
    # Remove missing values
    # Filtering: no missing values in ST's T, radiation and q
    mask=np.zeros(st2d.shape[1],dtype='int')
    for i in range(len(mask)):
        if np.isnan(st2d[1,i])==False and np.isnan(st2d[2,i])==False and np.isnan(st2d[5,i])==False and np.isnan(the1d_0[i])==False:
           mask[i]+=1
    st1d=st2d[:,mask==1]
    st1d=pd.DataFrame(st1d)
    st1d.index=['P','T','RH','U','V','rad','q','th','the']
    st1d.T.to_csv(iopath+'STIO.csv',index=False)
    vs1d=vs2d[:,mask==1]
    vs1d=pd.DataFrame(vs1d)
    vs1d.index=['P','T','RH','q','th','the']
    vs1d.T.to_csv(iopath+'VSIO.csv',index=False)
    the1d=the1d_0[mask==1]
    return st1d.T, vs1d.T, the1d

In [7]:
def evaluation(model,x,y,model_name,var):
    print("===== RFE feature selection =====")
    #from sklearn.model_selection import StratifiedKFold
    from sklearn.feature_selection import RFECV
    # Create the RFE object and compute a cross-validation score.
    rfecv=RFECV(estimator=model, step=1,cv=10)
    rfecv.fit(x,y)
    print("Optimal number of features : %d" % rfecv.n_features_)

    ranking = rfecv.ranking_
    print(list(ranking))

    # Plot number of features VS. cross-validation scores
    #plt.figure()
    #plt.xlabel("Number of features selected")
    #plt.ylabel("Cross validation score")
    #plt.plot(range(1, len(t_rfecv.grid_scores_) + 1), t_rfecv.grid_scores_)
    #plt.savefig('glm_all_t_rfecv10.png')
    #plt.savefig('glm_noUV_t_rfecv10.png')
    #plt.close()

    #x_sel=list(x1[:,rfecv.support_])
    print(list(rfecv.support_))

    # Comprehensive evaluation
    glm_sm=sm.OLS(y,x)
    summary=glm_sm.fit(maxiter=30).summary()
    print(summary)

    #with open(model_name+'_summary.txt','w') as f:
    #     for lines in summary:
    #         lines.write('%s' %line)

    # P-value and F-score
    f_score, p_val=f_regression(x,y)
    print('F-value: '+str(f_score))
    print('p-value: '+str(p_val))

    f_sort=f_score
    print(var.shape)
    v_sort=np.array(var)
    for j in range(len(f_score)):
        for i in range(len(f_score)-1):
            if f_score[i] > f_score[i+1]:
               tmp=f_sort[i]
               f_sort[i]=f_sort[i+1]
               f_sort[i+1]=tmp

               v_tmp=v_sort[i]
               v_sort[i]=v_sort[i+1]
               v_sort[i+1]=v_tmp

    fscore_df=pd.DataFrame(f_sort)
    fscore_df.index=v_sort

    df=pd.DataFrame()
    df.insert(0,'Ranking',ranking,True)
    df.insert(1,'Support',rfecv.support_,True)
    df.insert(2,'F-score',f_score,True)
    df.insert(3,'p-value',p_val,True)
    df.index=var
    #df.to_csv(model_name+'_score.csv')

    # Plotting
    fsize=18
    y_pos=np.arange(len(f_score))
    plt.figure(figsize=(12,7))
    plt.barh(y_pos,f_sort,align='center',height=0.2)
    plt.grid()
    plt.yticks(y_pos,v_sort,fontsize=fsize)
    plt.xticks(fontsize=fsize)
    plt.xlabel('F-value',fontsize=fsize)
    plt.ylabel('Features',fontsize=fsize)
    #plt.title(model_name+' model feature importance',fontsize=12)
    plt.tight_layout()
    plt.savefig(iopath+model_name+'_score.png')
    plt.close()

def glm(x,y):
    print('===== Training Linear Regression Model  =====')
    model=LinearRegression().fit(x,y)
    print(model.score(x,y))
    coef={'GLM_thetae_coef':model.coef_}
    dft=pd.DataFrame(coef)
    dft.index=['P','T','RH','U','V','rad','q','th','the']
    dft.to_csv(iopath+'glm_thetae_coef.csv', header=None)

    # Save the model as a pickle object
    with open(iopath+'glm_thetae.pkl','wb') as f:
         pickle.dump(model,f)
    evaluation(model,x,y,'glm_thetae',x.columns)
    del model
    return y

def predict(x):
    with open(iopath+'glm_thetae.pkl','rb') as f:
         model=pickle.load(f)
    y_pred=model.predict(x)
    return y_pred

In [8]:
class FCNN(nn.Module):
  def __init__(self, input_dim):
    super(FCNN,self).__init__()
    self.fc1=nn.Linear(input_dim,64)
    self.fc2=nn.Linear(64,32)
    self.fc3=nn.Linear(32,16)
    self.output = nn.Linear(16, 1)

  def forward(self, x):
    x=F.relu(self.fc1(x))
    x=F.relu(self.fc2(x))
    x=F.relu(self.fc3(x))
    x = self.output(x)

    return x
summary(FCNN(9),input_size=(1,9),batch_size=50,device="cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [50, 1, 64]             640
            Linear-2                [50, 1, 32]           2,080
            Linear-3                [50, 1, 16]             528
            Linear-4                 [50, 1, 1]              17
Total params: 3,265
Trainable params: 3,265
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.04
Params size (MB): 0.01
Estimated Total Size (MB): 0.06
----------------------------------------------------------------


In [9]:
def loss_fn(y_pred, y):
    mse = torch.nn.functional.mse_loss(y_pred, y)
    return mse

In [10]:
def scoring(y_pred,st_the,vs_the):
    y=(st_the*100.+300.)+y_pred
    vs_the=vs_the*100.+300.
    rmse=np.sqrt(mean_squared_error(vs_the,y))
    corr=np.corrcoef(vs_the,y)[0,1]
    print('Correlation: '+str(corr))
    print('RMSE: '+str(rmse))
    return corr,rmse

In [17]:
def main():
    # Load ST and VS iodata
    st_train, st_test, vs_train, vs_test, the_diff_train, the_diff_test, date_train, date_test=load_data('/content/drive/MyDrive/MLQC/st_io_1hPa.csv'
          ,'/content/drive/MyDrive/MLQC/vs_io_1hPa.csv')

    print("Training dataset:")
    st_train_1d, vs_train_1d, the_diff_train_1d=nd21d(st_train, vs_train, the_diff_train)
    print(st_train_1d.shape)
    print(vs_train_1d.shape)
    print(the_diff_train_1d.shape)
    print("Testing dataset:")
    st_test_1d, vs_test_1d, the_diff_test_1d=nd21d(st_test,vs_test,the_diff_test)
    print(st_test_1d.shape)
    print(vs_test_1d.shape)
    print(the_diff_test_1d.shape)

    # Train GLM
    #y_pred_train=glm(st_train_1d,the_diff_train_1d)
    #scoring(y_pred_train,st_train_1d['the'],vs_train_1d['the'])

    # Test GLM
    #y_pred=predict(st_test_1d)
    #scoring(y_pred,st_test_1d['the'],vs_test_1d['the'])

    # Training NN model
    print("===== Training Fully-connected NN T-model =====")

    # Use GPU
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)

    # Hyperparameters
    learning_rate = 0.001
    epochs = 100
    batch_size = 50
    nvar = 9
    epo_print=1

    #  Convert to torch object
    input_data=torch.from_numpy(st_train_1d.values).float()
    target_data=torch.from_numpy(the_diff_train_1d).float()

    #  Dataloader
    dataset = TensorDataset(input_data, target_data)
    train_loader = DataLoader(dataset = dataset, batch_size = batch_size, shuffle = True)

    # Load Fully-connected NN model to gpu
    fcnn_t = FCNN(input_dim=nvar).to(device)
    fcnn_t.train()

    # Optimizer
    optimizer = torch.optim.Adam(fcnn_t.parameters(), lr=learning_rate)

    # Training
    e_loss=[]
    min_loss = 99999999999

    for epoch in range(epochs):
        tot_loss=0.
        for batch_inputs, batch_targets in train_loader:
            batch_inputs=batch_inputs.to(device)
            batch_targets=batch_targets.to(device)
            fcnn_t.zero_grad()
            outputs = fcnn_t(batch_inputs)
            #print(outputs[:,0],batch_targets)

            loss = loss_fn(outputs[:,0], batch_targets)
            loss.backward()
            optimizer.step()
            tot_loss+=loss.item()

        tot_loss=tot_loss/len(train_loader.dataset)


        if epoch % epo_print ==0:
            e_loss.append(tot_loss)
            updated = False
            if min_loss > tot_loss:
                updated = True
                min_loss = tot_loss
                torch.save(fcnn_t, iopath+'FCNN_THE.pkl')
            print (
            '[{:>5d}/{:>5d}]'.format(epoch+1, epochs),
            'Loss:{:>.8e}, '.format(tot_loss),
            'updated = {:>6s}, min loss={:>.8e}'.format(str(updated),min_loss)
            )

    del fcnn_t

    # Testing NN model
    print("===== Testing Fully-connected NN Theta_e-model =====")
    fcnn_t=torch.load(iopath+'FCNN_THE.pkl').to(device)
    fcnn_t.eval()
    input_data=input_data.to(device)
    output=fcnn_t(input_data)
    output=output.detach().cpu().numpy()
    print(output.shape)
    scoring(output[:,0],st_train_1d['the'],vs_train_1d['the'])

    # Plot

In [18]:
if __name__=='__main__':
   main()

Training dataset:
(471791, 9)
(471791, 6)
(471791,)
Testing dataset:
(116492, 9)
(116492, 6)
(116492,)
===== Training Fully-connected NN T-model =====
cpu


KeyboardInterrupt: 